# Evaluation code

### GPU 

In [ ]:
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = "1" #please put your GPU

## Importing libraries 

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import Sequential, layers
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam, SGD
from io_ import score
import matplotlib.pyplot as plt

In [ ]:
tf.__version__

## Splitting data into train, test, val 

In [ ]:
# data_dir = 'dataset'

In [ ]:
# !pip install split-folders tqdm

In [ ]:
# import splitfolders  # or import split_folders  #split into train, test, val

# # Split with a ratio.
# # To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
# splitfolders.ratio(data_dir, output="Splitteddata/", seed=1337, ratio=(.8, .1, .1), group_prefix=None) # default values


## Test data directory

In [ ]:
test_dir = 'Splitteddata/test'

## Data preprocessing 

In [ ]:
batch_size = 128
img_height = 225
img_width = 225

In [ ]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  test_dir,
  seed=123,
  image_size=(img_height, img_width),
  label_mode = 'int',                  #sparse categorical loss
  batch_size=batch_size)

In [ ]:
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

In [ ]:
normalized_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixels values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

In [ ]:
test_ds = normalized_ds

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in test_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy())

# Score each of the images based on the likelihood it belongs to a class. Larger scores indicate higher likelihood.

Classes are arranged alphabeticall, [1_Capitol, 1_Chijmes,..., 1_scis, americano, ... taco]. Screen-shot is attached.

The first thirteen belong to buidings and the next fifty nine belong to food, this makes for a total of 72 classes.

Scores should take the form of a [Nx72] array, where N is the number of test images.


## Detecting number of images in test folder (required to create matrix later) 

In [ ]:
# num_images = len(np.concatenate([i for x, i in test_ds], axis=0))

In [ ]:
num_images = 3082  #change accordingly   #hard-coded  #run this if cell above takes too long

## Importing test model

In [ ]:
# Model reconstruction from JSON file
with open('Siti(27).json', 'r') as json_file:
    json_savedModel= json_file.read()

test_model = tf.keras.models.model_from_json(json_savedModel)
test_model.summary()

optim = SGD(lr=0.01, momentum=0.9)

# test_model.compile(loss='sparse_categorical_crossentropy',
#               optimizer=optim,
#               metrics=[tf.keras.metrics.SparseTopKCategoricalAccuracy(k = 1)])

test_model.compile(loss='sparse_categorical_crossentropy',
              optimizer=optim,
              metrics=['accuracy', tf.keras.metrics.SparseTopKCategoricalAccuracy(k = 5)])

# Load weights into the new model
test_model.load_weights('Siti(27).h5')

## Creating matrix for ground truth

In [ ]:
ground_truth = np.zeros([num_images])

In [ ]:
i = 0
for images, labels in test_ds:
    
    print(labels)
    labels_array = labels.numpy()
    print(labels_array)
    for lbl in labels_array:
        ground_truth[i] = lbl
        i+=1
   

In [ ]:
len(ground_truth)

In [ ]:
ground_truth

In [ ]:
ground_truth = ground_truth.astype('int')

In [ ]:
ground_truth

## Forming matrix for predicted scores

In [ ]:
predicted_scores = np.zeros([num_images, 72])

In [ ]:
i = 0
for images, labels in test_ds:
    img_scores = test_model.predict(images)
    print(img_scores)
    for score in img_scores:
        print(score)
        predicted_scores[i] = score
        i += 1



In [ ]:
np.sum(predicted_scores[0])

In [ ]:
# import matplotlib.pyplot as plt
# plt.imshow(img_view[0].numpy().astype("uint8"))

## Obtaining accuracy (Comparing ground truth with predicted scores)

In [ ]:
from io_ import score
print('overall accuracy')
scores = predicted_scores
gt = ground_truth 
top1, top5 = score(scores, gt, 5)
print('percentage top1 accuracy:', top1)
print('percentage top5 accuracy:', top5)


## Demonstrating the evaluation functions on the first two test images

You need not modify code beyond this point.

In [ ]:
from io_ import score
print('overall accuracy')

# simulated scores
scores = np.zeros([2,72])
scores[0,0] = 1000
scores[1,3] = 1000
scores[1,21] = 999

# Ground-truth: the first image belongs to class 0; the second image to class 13.
gt = np.array([0,21]) 

top1, top5 = score(scores, gt, 5)
print('percentage top1 accuracy:', top1)
print('percentage top5 accuracy:', top5)

In [ ]:
print('accuracy on building only')
mask = gt<13
top1, top5 = score(scores[mask,:13], gt[mask], 5)
print('percentage top1 accuracy:', top1)
print('percentage top5 accuracy:', top5)

In [ ]:
print('accuracy on food only')
mask = gt>=13
top1, top5 = score(scores[mask,13:], gt[mask]-13, 5)
print('percentage top1 accuracy:', top1)
print('percentage top5 accuracy:', top5)